<a href="https://colab.research.google.com/github/sundarp17/sundar_info5731_fall2020/blob/master/assignment4_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import json

In [4]:
import zipfile
with zipfile.ZipFile('/content/Assignment four data Yelp (question 1 and 2).zip', 'r') as zip_ref:
    zip_ref.extractall('/content/Assignment4')

In [20]:
reviews=[]
import glob
for files in glob.iglob('/content/Assignment4/Assignment four data Yelp (question 1 and 2)/*'):
  f=open(files)
  data=json.load(f)
  for i in data:
    reviews.append(i['text'])

print("total reviews:",len(reviews))


total reviews: 500000


In [24]:
ratings=[]
for files in glob.iglob('/content/Assignment4/Assignment four data Yelp (question 1 and 2)/*'):
  f=open(files)
  data=json.load(f)
  for i in data:
    ratings.append(i['stars'])

print("total ratings:",len(ratings))


total ratings: 500000


In [25]:
import pandas as pd

df = pd.DataFrame(reviews,columns=['review'])
df['ratings']=ratings
df.head()

,review,ratings
0,"Wow, called them at 7am because my RO system w...",5.0
1,Good take out with large portions. Thai tea wa...,4.0
2,"Every experience thusfar, and there have been ...",5.0
3,"First time at the Stone and Vine, very good fo...",4.0
4,"Great delicious sammiches, coffee, and treats....",5.0


In [26]:
df['ratings']=df['ratings'].astype(int)
df.head()

,review,ratings
0,"Wow, called them at 7am because my RO system w...",5
1,Good take out with large portions. Thai tea wa...,4
2,"Every experience thusfar, and there have been ...",5
3,"First time at the Stone and Vine, very good fo...",4
4,"Great delicious sammiches, coffee, and treats....",5


In [28]:
df['ratings'].unique()

array([5, 4, 3, 2, 1])

Cleaning


In [27]:
import re
#removing punctutations
df['review']=df['review'].map(lambda x: re.sub('[?|!|\'|"|#]', '', x))
#lower
df['review']=df['review'].map(lambda x: x.lower())

df.head(10)

,review,ratings
0,"wow, called them at 7am because my ro system w...",5
1,good take out with large portions. thai tea wa...,4
2,"every experience thusfar, and there have been ...",5
3,"first time at the stone and vine, very good fo...",4
4,"great delicious sammiches, coffee, and treats....",5
5,"i brought a group of 20 dentists here, and we ...",5
6,my only complaint is that they dont have locat...,5
7,theyre lunch prices are awesome. the amount of...,5
8,this place is really nice tucked inside treasu...,4
9,had a great time tonight the food like always ...,5


In [29]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
words = stopwords.words('english')
from textblob import Word
from textblob import TextBlob
nltk.download('punkt')
nltk.download('wordnet')

df['review']=df['review'].apply(lambda x:" ".join(x for x in x.split() if x not in words))
df['review']=df['review'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

df.head(10)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,review,ratings
0,"wow, called 7am ro system constantly running a...",5
1,good take large portions. thai tea excellent l...,4
2,"every experience thusfar, several, 5 star fron...",5
3,"first time stone vine, good food atmosphere go...",4
4,"great delicious sammiches, coffee, treats. eve...",5
5,"brought group 20 dentist here, amazing time. a...",5
6,complaint dont location colorado place great f...,5
7,theyre lunch price awesome. amount food get $1...,5
8,place really nice tucked inside treasure islan...,4
9,great time tonight food like always amazing be...,5


supervisied LDA

In [49]:
!pip install tomotopy

     |████████████████████████████████| 13.5MB 318kB/s 
     |████████████████████████████████| 102kB 12.8MB/s 
  Created wheel for py-cpuinfo: filename=py_cpuinfo-7.0.0-cp36-none-any.whl size=20071 sha256=4e903906d845901004a01fbe9933e7aa708e617abe7d4d31582384129b570804
  Stored in directory: /root/.cache/pip/wheels/f1/93/7b/127daf0c3a5a49feb2fecd468d508067c733fba5192f726ad1
Successfully built py-cpuinfo


In [57]:
import os
import logging
import nltk
import numpy as np
import matplotlib.pyplot as plt

import tomotopy as tp
mdl = tp.SLDAModel(k=20) #supervisedLDA
for i in df['review']:
    mdl.add_doc(i.split())
for i in range(0, 100, 10):
    mdl.train(10)
    print('Iteration: {}\tLog-likelihood: {}'.format(i, mdl.ll_per_word))
for k in range(mdl.k):
    print('Top 15 words of topic #{}'.format(k))
    print(mdl.get_topic_words(k, top_n=15))


Iteration: 0	Log-likelihood: -10.25496379458646
Iteration: 10	Log-likelihood: -9.734372019679011
Iteration: 20	Log-likelihood: -9.499059523878062
Iteration: 30	Log-likelihood: -9.374299705798231
Iteration: 40	Log-likelihood: -9.302230465976654
Iteration: 50	Log-likelihood: -9.258518728834002
Iteration: 60	Log-likelihood: -9.230839842661917
Iteration: 70	Log-likelihood: -9.212302650142698
Iteration: 80	Log-likelihood: -9.19835605201983
Iteration: 90	Log-likelihood: -9.187879632875875
Top 15 words of topic #0
[('steak', 0.009615588933229446), ('restaurant', 0.007514442317187786), ('wine', 0.006365377921611071), ('dinner', 0.0062333098612725735), ('-', 0.00599678186699748), ('salad', 0.005752792116254568), ('ordered', 0.005342411808669567), ('meal', 0.00527302036061883), ('great', 0.005267051048576832), ('bread', 0.005238697398453951), ('dish', 0.0051685599610209465), ('best', 0.005166321527212858), ('cheese', 0.005144683178514242), ('dessert', 0.005018584430217743), ('rib', 0.00465222680

Topic 0: It is a review about a restaurant and types of food items available like wine, steak, salads.
---------------------------------------------------------------
Topic 1: Topic 1 contains regular words like 'know, back, even,etc..'. So, I can't phrase a sentence from it.
---------------------------------------------------------------
Topic 2: The customer ordered a seafood item. It may be a sushi roll and fried shrimp and tuna with extra spice.
---------------------------------------------------------------
Topic 3: It contains some spanish words.
---------------------------------------------------------------
Topic 4: The room in the casino hotel was nice and the hotel contains a pool.
--------------------------------------------------------------
Topic 5: The server didn't bring the ordered food to the table.
---------------------------------------------------------------
Topic 6: The waiter came back in one minute with food after taking the order.
---------------------------------------------------------------
Topic 7: It is about the how good was the breakfast. The customer ordered pizza and a chicken sandwich with cheese and egg.
---------------------------------------------------------------
Topic 8: The customer called the service number.
---------------------------------------------------------------
Topic 9: This is the best place for food.
---------------------------------------------------------------
Topic 10: Find the parking near the store location.
---------------------------------------------------------------
Topic 11: It is about a salon shop and how great the services provided there like the massage, haircut.
---------------------------------------------------------------
Topic 12: Good restaurant with several types of dishes such as chicken, rice, thai, chinese,noodles,etc..
---------------------------------------------------------------
Topic 13: The bar was a good place to drink, to have fun and enjoy the music.
---------------------------------------------------------------
Topic 14: The bar was a good place to have beer and food.
---------------------------------------------------------------
Topic 15: The staff was great and I had great experience and I would recommend everyone.
---------------------------------------------------------------
Topic 16: It was a great place to have food and I love the service provided by the friendly staff.
---------------------------------------------------------------
Topic 17: I would like to order Iced coffee tea and a choclate cake.
---------------------------------------------------------------
Topic 18: The new guy at the car service is doing great.
---------------------------------------------------------------
Topic 19: The customer ordered chicken burger and a cheese sandwich which was good.
---------------------------------------------------------------



In [59]:
!pip install pyLDAvis
import numpy as np
import pyLDAvis


topic_term_dists = np.stack([mdl.get_topic_word_dist(k) for k in range(mdl.k)])
doc_topic_dists = np.stack([doc.get_topic_dist() for doc in mdl.docs])
doc_lengths = np.array([len(doc.words) for doc in mdl.docs])
vocab = list(mdl.used_vocabs)
term_frequency = mdl.used_vocab_freq

prepared_data = pyLDAvis.prepare(
    topic_term_dists, 
    doc_topic_dists, 
    doc_lengths, 
    vocab, 
    term_frequency
)
pyLDAvis.save_html(prepared_data, 'ldavis.html')

     |████████████████████████████████| 1.6MB 13.6MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97712 sha256=4c93e4356d9be1f1715bc2a8f8f7e5fbc607b337d01ca5c4f7091ef5781dfd97
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyLDAvis


In [77]:
model = tp.LLDAModel(k=20, min_cf=100, rm_top=200)

for i in range(len(df)):
  content = df['review'][i]
  label =df['ratings'][i].astype('str')
  clean_document = [token for token in content.strip().split() if len(token) > 2]
  model.add_doc(clean_document, labels=label)

for i in range(0, 100, 10):
    model.train(10)
    print('Iteration: {}\tLog-likelihood: {}'.format(i, model.ll_per_word))
for k in range(model.k):
    print('Top 15 words of global topic #{} {}: {}'.format(k, model.topic_label_dict[k], model.get_topic_words(k, top_n=15)))


Iteration: 0	Log-likelihood: -8.553474290335911
Iteration: 10	Log-likelihood: -8.552395206385354
Iteration: 20	Log-likelihood: -8.552079833306664
Iteration: 30	Log-likelihood: -8.552286638734147
Iteration: 40	Log-likelihood: -8.552241814328756
Iteration: 50	Log-likelihood: -8.552262576887824
Iteration: 60	Log-likelihood: -8.552182701585659
Iteration: 70	Log-likelihood: -8.552051709091009
Iteration: 80	Log-likelihood: -8.552061055297239
Iteration: 90	Log-likelihood: -8.552095003041511
Top 15 words of global topic #0 5: [('perfect', 0.0022761954460293055), ('thank', 0.0020793257281184196), ('absolutely', 0.0019637488294392824), ('delicious.', 0.0019094949821010232), ('wonderful', 0.0018471116200089455), ('amazing.', 0.0018446375615894794), ('spot', 0.0017262330511584878), ('job', 0.0017089141765609384), ('owner', 0.0016864703502506018), ('well.', 0.0016541300574317575), ('fun', 0.0016214362112805247), ('must', 0.001557285781018436), ('able', 0.0015260057989507914), ('atmosphere', 0.00152